In [65]:
import numpy as np
import pandas as pd
import glob, os 

## Initial data import
Use data starting in 2000

In [66]:
pathname = "./tennis_atp/atp_matches_20??.csv"
df = pd.concat((pd.read_csv(f) for f in glob.iglob(pathname, recursive=True)), ignore_index=True)
# Filter matches without data
df = df[list(map(lambda x: not np.isnan(x), df.loc[:]["minutes"]))]

df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2000-717,Orlando,Clay,32.0,A,2000-05-01,1.0,102179.0,NaN,NaN,...,15.0,13.0,4.0,110.0,59.0,49.0,31.0,17.0,4.0,4.0
1,2000-717,Orlando,Clay,32.0,A,2000-05-01,2.0,103602.0,NaN,Q,...,6.0,0.0,0.0,57.0,24.0,13.0,17.0,10.0,4.0,9.0
2,2000-717,Orlando,Clay,32.0,A,2000-05-01,3.0,103387.0,NaN,NaN,...,0.0,2.0,2.0,65.0,39.0,22.0,10.0,8.0,6.0,10.0
3,2000-717,Orlando,Clay,32.0,A,2000-05-01,4.0,101733.0,NaN,NaN,...,12.0,4.0,6.0,104.0,57.0,35.0,24.0,15.0,6.0,11.0
4,2000-717,Orlando,Clay,32.0,A,2000-05-01,5.0,101727.0,4.0,NaN,...,1.0,0.0,3.0,47.0,28.0,17.0,10.0,8.0,3.0,6.0


In [67]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

## Match only

In [22]:
games = df[['tourney_id', 'winner_id', 'winner_rank', 'loser_id', 'loser_rank', 'tourney_date']].copy()
def label_data(winner_id, winner_rank, loser_id, loser_rank):
    if winner_rank < loser_rank:
        top_player_rank = winner_rank
        top_player_id = winner_id
        low_player_rank = loser_rank
        low_player_id = loser_id
        top_player_won = 1.
    else:
        top_player_rank = loser_rank
        top_player_id = loser_id
        low_player_rank = winner_rank
        low_player_id = winner_id
        top_player_won = 0.
    return top_player_id, top_player_rank, low_player_id, low_player_rank, top_player_won

games['top_player_id'], games['top_player_rank'], games['low_player_id'], games['low_player_rank'], games['top_player_won'] = \
np.vectorize(label_data)(games['winner_id'], games['winner_rank'], games['loser_id'], games['loser_rank'])
games.drop(['winner_id', 'winner_rank', 'loser_id', 'loser_rank'], axis=1, inplace=True)
games.head()
        

,tourney_id,tourney_date,top_player_id,top_player_rank,low_player_id,low_player_rank,top_player_won
0,2004-360,20040517.0,103103.0,32.0,104259.0,149.0,1.0
1,2004-360,20040517.0,103153.0,101.0,102231.0,150.0,0.0
2,2004-360,20040517.0,103700.0,136.0,102558.0,165.0,1.0
3,2004-360,20040517.0,103169.0,90.0,103007.0,451.0,1.0
4,2004-360,20040517.0,103105.0,80.0,103898.0,109.0,0.0


### Database by player

In [29]:
winners = (df[list(df.columns[:7]) + [column for column in df.columns if column[0]=='w'] + ['l_bpFaced', 'l_bpSaved']]
               .rename(columns={'l_bpFaced': '_break_points_for', 'l_bpSaved':'_break_points_missed'})
          )

for column in winners.columns[7:]:
    index = column.find('_') + 1
    winners.rename(columns={column: column[index:]}, inplace = True)
    
losers = (df[list(df.columns[:7]) + [column for column in df.columns if column[0]=='l'] + ['w_bpFaced', 'w_bpSaved']]
               .rename(columns={'w_bpFaced': '_break_points_for', 'w_bpSaved':'_break_points_missed'})
          )
for column in losers.columns[7:]:
    index = column.find('_') + 1
    losers.rename(columns={column: column[index:]}, inplace = True)

players = pd.DataFrame.append(winners, losers)
players['id'] = players['id'].astype('int')

players['%1st_serve_in'] = players['1stIn']/players['svpt']
players['%1st_serve_won'] = players['1stWon']/players['1stIn']
players['%2nd_serve_won'] = players['2ndWon']/(players['svpt'] - players['1stIn'])
players['%break_points_saved'] = players['bpSaved']/players['bpFaced']
players['%break_points_converted'] = 1. - players['break_points_missed']/players['break_points_for']

players.rename(columns={
    'df': 'double_fault',
    'bpFaced': 'break_points_against'
}, inplace=True)

players.head()

  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  \
0   2004-360   Casablanca    Clay       32.0             A    20040517.0   
1   2004-360   Casablanca    Clay       32.0             A    20040517.0   
2   2004-360   Casablanca    Clay       32.0             A    20040517.0   
3   2004-360   Casablanca    Clay       32.0             A    20040517.0   
4   2004-360   Casablanca    Clay       32.0             A    20040517.0   

   match_num      id  seed entry         ...            df  svpt  1stIn  \
0        1.0  103103   1.0   NaN         ...           1.0  56.0   32.0   
1        2.0  102231   NaN   NaN         ...           4.0  83.0   59.0   
2        3.0  103700   NaN   NaN         ...           2.0  50.0   26.0   
3        4.0  103169   8.0   NaN         ...           4.0  60.0   39.0   
4        5.0  103898   NaN   NaN         ...           5.0  73.0   43.0   

  1stWon  2ndWon  SvGms  bpSaved  bpFaced  break_points_for  \
0   27.0    11.0    8.0      

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,SvGms,bpSaved,break_points_against,break_points_for,break_points_missed,%1st_serve_in,%1st_serve_won,%2nd_serve_won,%break_points_saved,%break_points_converted
0,2004-360,Casablanca,Clay,32.0,A,20040517.0,1.0,103103,1.0,NaN,...,8.0,6.0,6.0,9.0,5.0,0.571429,0.843750,0.458333,1.000000,0.444444
1,2004-360,Casablanca,Clay,32.0,A,20040517.0,2.0,102231,NaN,NaN,...,14.0,6.0,11.0,13.0,6.0,0.710843,0.559322,0.500000,0.545455,0.538462
2,2004-360,Casablanca,Clay,32.0,A,20040517.0,3.0,103700,NaN,NaN,...,7.0,3.0,3.0,8.0,3.0,0.520000,0.769231,0.541667,1.000000,0.625000
3,2004-360,Casablanca,Clay,32.0,A,20040517.0,4.0,103169,8.0,NaN,...,10.0,3.0,4.0,20.0,17.0,0.650000,0.692308,0.619048,0.750000,0.150000
4,2004-360,Casablanca,Clay,32.0,A,20040517.0,5.0,103898,NaN,NaN,...,9.0,6.0,7.0,8.0,4.0,0.589041,0.674419,0.533333,0.857143,0.500000


## Statistics

In [70]:
stats = ['ace', 'double_fault', '%1st_serve_in', '%1st_serve_won', '%2nd_serve_won', '%break_points_saved', 'break_points_against', '%break_points_converted']

### Life to date

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,SvGms,bpSaved,break_points_against,break_points_for,break_points_missed,%1st_serve_in,%1st_serve_won,%2nd_serve_won,%break_points_saved,%break_points_converted
0,2004-360,Casablanca,Clay,32.0,A,20040517.0,1.0,103103,1.0,NaN,...,8.0,6.0,6.0,9.0,5.0,0.571429,0.843750,0.458333,1.000000,0.444444
1,2004-360,Casablanca,Clay,32.0,A,20040517.0,2.0,102231,NaN,NaN,...,14.0,6.0,11.0,13.0,6.0,0.710843,0.559322,0.500000,0.545455,0.538462
2,2004-360,Casablanca,Clay,32.0,A,20040517.0,3.0,103700,NaN,NaN,...,7.0,3.0,3.0,8.0,3.0,0.520000,0.769231,0.541667,1.000000,0.625000
3,2004-360,Casablanca,Clay,32.0,A,20040517.0,4.0,103169,8.0,NaN,...,10.0,3.0,4.0,20.0,17.0,0.650000,0.692308,0.619048,0.750000,0.150000
4,2004-360,Casablanca,Clay,32.0,A,20040517.0,5.0,103898,NaN,NaN,...,9.0,6.0,7.0,8.0,4.0,0.589041,0.674419,0.533333,0.857143,0.500000


### Last x games

In [78]:
def last_x_games_avg(df, player_id, tdate, x):
    column_names = ['last_' + str(x) + '_games_' + stat for stat in stats]
    results = (df.loc[(df['id'] == player_id) & (df['tourney_date'] < tdate)]
                    .sort_values(by=['tourney_date'], ascending=False)
                    .head(x)
                    .agg('mean')[stats]
                    .rename(columns=dict(zip(stats, column_names)))
              )
    return results

#last_x_games_avg(players, 104735, '2016-01-04', 10)
np.vectorize(last_x_games_avg, excluded=['df', 'x'])(df=players, player_id=players['id'], tdate=players['tourney_date'], x=10)


TypeError: <class 'int'> type object 957139200000000000

### Current year

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,id,seed,entry,...,SvGms,bpSaved,bpFaced,bpOcc,bpMissed,%1stIn,%1stWon,%2ndWon,%bpSaved,%bpConverted
0,2000-717,Orlando,Clay,32.0,A,2000-05-01,1.0,102179,NaN,NaN,...,16.0,14.0,15.0,4.0,4.0,0.603175,0.736842,0.580000,0.933333,0.000000
1,2000-717,Orlando,Clay,32.0,A,2000-05-01,2.0,103602,NaN,Q,...,10.0,4.0,6.0,9.0,4.0,0.522388,0.714286,0.500000,0.666667,0.555556
2,2000-717,Orlando,Clay,32.0,A,2000-05-01,3.0,103387,NaN,NaN,...,8.0,0.0,0.0,10.0,6.0,0.630435,0.793103,0.647059,NaN,0.400000
3,2000-717,Orlando,Clay,32.0,A,2000-05-01,4.0,101733,NaN,NaN,...,15.0,9.0,12.0,11.0,6.0,0.513761,0.767857,0.396226,0.750000,0.454545
4,2000-717,Orlando,Clay,32.0,A,2000-05-01,5.0,101727,4.0,NaN,...,9.0,1.0,1.0,6.0,3.0,0.540000,0.814815,0.695652,1.000000,0.500000


### Face to face

In [ ]:
# Add number of wins to stats